In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

with open('../input/pretrained-for-riiid/user_ele_dict.pkl', 'rb') as f:
    user_ele_dict = pickle.load(f)

with open('../input/pretrained-for-riiid/user_num_dict.pkl', 'rb') as f:
    user_num_dict = pickle.load(f)

with open('../input/pretrained-for-riiid/ques_ele_vec.pkl', 'rb') as f:
    ques_ele_vec = pickle.load(f)

with open('../input/pretrained-for-riiid/ques_num_vec.pkl', 'rb') as f:
    ques_num_vec = pickle.load(f)  

In [5]:
question_dtype = {
    'question_id':'int16',
    'tags':'object'
}
questions_data = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',
                             usecols = question_dtype.keys(), 
                             dtype = question_dtype)

questions_data.tags.fillna('92',inplace=True)

def gen_vec(row):
    row['vec'] = np.zeros(188)
    index_list = row.tags.split()
    for index_ in index_list:
        row.vec[int(index_)] = 1.0
    return row

questions_data = questions_data.apply(gen_vec, axis='columns')

In [6]:
questions_data

,question_id,tags,vec
0,0,51 131 162 38,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,131 36 81,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,131 101 162 92,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,131 149 162 29,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,131 5 162 38,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
13518,13518,14,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
13519,13519,8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
13520,13520,73,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
13521,13521,125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
data = pd.read_pickle("../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip")
data['prior_question_had_explanation'] = data['prior_question_had_explanation'].astype('bool')
data = data[data.content_type_id == 0]
data.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,False
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False


In [17]:
len(data)

99271300

In [19]:
users_i_counts = data.user_id.value_counts()
user_list = users_i_counts.keys()
print(user_list[:5])
print(users_i_counts.head())

Int64Index([801103753, 1478712595, 455973631, 1842816145, 1743444187], dtype='int64')
801103753     17609
1478712595    16842
455973631     16640
1842816145    16533
1743444187    16424
Name: user_id, dtype: int64


In [25]:
len(user_list)

393656

In [31]:
sum(users_i_counts.values>10)

390686

In [28]:
users_i_counts.values[390685]

11

In [57]:
ave = ques_ele_vec/ques_num_vec
def pre_data(user_list_, ele_dict=user_ele_dict, num_dict=user_num_dict, ave=ave,q_data=questions_data,data=data):
    m = len(user_list_)
    X = np.zeros((m,188*3))
    y = np.zeros(m)
    i = 0
    for user in user_list_:
        user_data = data[data.user_id==user]
        mask = q_data.vec[user_data.content_id.values[-1]]
        X[i,:188] = np.nan_to_num(ele_dict[user]/num_dict[user],nan=0.25)
        X[i,188:188*2] = mask
        X[i,188*2:] = ave
        y[i] = user_data.answered_correctly.values[-1]
        i = i+1
    return X, y

NameError: name 'user_ele_dict' is not defined

In [10]:
user_num = 2 #control how many user to train
valid_num = 2 #control how many valid data to keep
train_data = pd.concat([data_q[data_q.user_id==u][:-valid_num] for u in user_list[:user_num]], axis=0)
valid_data = pd.concat([data_q[data_q.user_id==u][-valid_num:] for u in user_list[:user_num]], axis=0)

In [11]:
train_data

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
54058,54058,0,1283420,3822,0,1
54059,54059,29795,1283420,638,0,1
54060,54060,91982,1283420,4231,0,0
54061,54061,117342,1283420,7978,0,0
54062,54062,145733,1283420,8225,0,1
...,...,...,...,...,...,...
8331,8331,14243544102,24418,8895,0,0
8332,8332,14243573690,24418,5016,0,1
8333,8333,14243646189,24418,4673,0,0
8334,8334,14243678701,24418,4174,0,1


In [12]:
valid_data

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
61625,61625,22373215518,1283420,3400,0,0
61626,61626,22373215518,1283420,3399,0,1
8336,8336,14243703953,24418,4612,0,1
8337,8337,14243735782,24418,4163,0,1


In [13]:
train_data[7472:7476]

,row_id,timestamp,user_id,content_id,content_type_id,answered_correctly
61623,61623,22364257029,1283420,2039,0,1
61624,61624,22373215518,1283420,3401,0,1
1874,1874,0,24418,7900,0,0
1875,1875,24224,24418,7876,0,0


In [14]:
def cal_vec(train_row,ele_dict,num_dict,q_data=questions_data):
    num_dict[train_row.user_id] += q_data.vec[train_row.content_id]
    ele_dict[train_row.user_id] += q_data.vec[train_row.content_id] * train_row.answered_correctly

In [ ]:
user_ele_dict = dict()
user_num_dict = dict()
#ques_ele_vec = np.zeros(188)
#ques_num_vec = np.zeros(188)
for index, row in train_data_q.iterrows():
    #ques_ele_vec += questions_data.vec[row.content_id] * row.answered_correctly
    #ques_num_vec += questions_data.vec[row.content_id]
    if row.user_id in user_ele_dict.keys():
        cal_vec(row,user_ele_dict,user_num_dict)
    else:
        user_ele_dict[row.user_id] = np.zeros(188)
        user_num_dict[row.user_id] = np.zeros(188)
        cal_vec(row,user_ele_dict,user_num_dict)

In [ ]:
X_valid, y_valid = pre_data(valid_data_q, user_ele_dict, user_num_dict)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# 188*[3,2,1] acc7.0slow |188*[2,1] acc7.0+ 20epoch|
model = keras.Sequential([
    layers.Dense(units=188*2,input_shape=[188*3], activation='relu'),
    layers.Dense(units=188, activation='relu'),
    layers.Dense(units=1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),#1e-2 can't descend 1e-4best
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
import math
import matplotlib.pyplot as plt
n = len(train_data_q)
batch_size = 256
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []
epochs = 20

for epoch in range(epochs):
    epoch_train_loss = []
    epoch_train_acc = []
    for batch_index in range(math.ceil(n/batch_size)):
        
        if batch_index*batch_size+batch_size < n:
            X_train, y_train = pre_data(train_data_q[batch_index*batch_size:batch_index*batch_size+batch_size], user_ele_dict, user_num_dict)
        else:
            X_train, y_train = pre_data(train_data_q[batch_index*batch_size:], user_ele_dict, user_num_dict)
        
        batch_loss = model.train_on_batch(X_train, y_train,reset_metrics=False,return_dict=True)
        epoch_train_loss.append(batch_loss['loss'])
        epoch_train_acc.append(batch_loss['binary_accuracy'])
        
    train_loss.append(np.mean(epoch_train_loss))
    train_acc.append(np.mean(epoch_train_acc))
    
    epoch_valid_loss = model.test_on_batch(X_valid, y_valid,reset_metrics=False, return_dict=True)
    valid_loss.append(epoch_valid_loss['loss'])
    valid_acc.append(epoch_valid_loss['binary_accuracy'])
    print("Epoch: ", epoch, 
          "train_loss: ", round(np.mean(epoch_train_loss),4),
          "valid_loss: ", round(epoch_valid_loss['loss'],4),
          "train_acc: ", round(np.mean(epoch_train_acc),4),
          "valid_acc: ", round(epoch_valid_loss['binary_accuracy'],4))

model.save("my_h5_model.h5")
print("model saved!")

In [ ]:
plt.plot(train_loss)
plt.plot(valid_loss)

In [ ]:
plt.plot(train_acc)
plt.plot(valid_acc)

In [ ]:
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_valid, y_valid),
#     batch_size=256,
#     epochs=20,
# )

In [ ]:
# # convert the training history to a dataframe
# history_df = pd.DataFrame(history.history)
# # use Pandas native plot method
# #history_df['loss'].plot();
# history_df.loc[:, ['loss', 'val_loss']].plot()
# history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
test_sample = train_data_q[70000:]
X_test, y_test = pre_data(test_sample, user_ele_dict, user_num_dict)

In [ ]:
predict_prob = model.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_prob)

In [ ]:
# def predict_y(row_data, user_ele, user_num, ques_ele=ques_ele_vec, ques_num=ques_num_vec, q_data=questions_data):
#     y = np.ones(len(row_data))*0.25
#     i = 0
#     for index, row in row_data.iterrows():
#         mask = q_data.vec[row.content_id]
#         ques_prob_vec = ques_ele/ques_num
#         if row.user_id in user_ele.keys():
#             borrow_index = (user_num[row.user_id] == 0)
#             user_prob_vec = user_ele[row.user_id]/user_num[row.user_id]
#             user_prob_vec[borrow_index] = ques_prob_vec[borrow_index]
#             y[i] = sum(user_prob_vec * mask)/sum(mask)
#         else:
#             y[i] = sum(ques_prob_vec * mask)/sum(mask)
#         i = i + 1
#     return y

In [ ]:
# import riiideducation
# env = riiideducation.make_env()
# iter_test = env.iter_test()
# for (test_df, sample_prediction_df) in iter_test:
#     test_df = test_df[test_df.content_type_id == 0]
#     test_df['answered_correctly'] = predict_y(test_df,user_ele_dict, user_num_dict)
#     #print('-----------------------------------------')
#     #print(test_df[['row_id', 'answered_correctly']])
#     #print('-----------------------------------------')
#     env.predict(test_df[['row_id', 'answered_correctly']])